# agent 配置

## 基础配置

In [ ]:
from agents import Agent, ModelSettings, function_tool, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv
from litellm import completion

# 加载环境变量
load_dotenv()

# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"

# api_key = os.getenv('DEEPSEEK_API_KEY')
# base_url = "https://api.deepseek.com/v1"
# chat_model = "deepseek/deepseek-chat"

In [ ]:
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

agent = Agent(
    name="天气助手",
    instructions="始终用汉赋的形式回答用户",
    model=llm,
    tools=[get_weather],  # 定义 tool
    model_settings=ModelSettings(temperature=0.2)
)

result = await Runner.run(agent, "上海最近适合出去玩吗?")
print(result.final_output)

吴淞江畔，春意盎然，上海近日天气宜人，阳光明媚，微风拂面，实为出游佳期。然需注意，偶有阵雨，建议携带雨具，以备不时之需。


## 输出类型控制



https://pydantic.com.cn/#pydantic_1

pydantic

In [4]:
from pydantic import BaseModel

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

class Candidate(BaseModel):
    name: str
    gender: str
    location: str

agent = Agent(
    name="简历助手",
    instructions="根据要求的格式抽取相应的信息",
    model=llm,
    output_type=Candidate,
)

result = await Runner.run(agent, "我叫李婷，女，现居上海，想要应聘前台岗位。")
print(result.final_output)

name='李婷' gender='女' location='上海'


## 上下文

In [ ]:
@dataclass
class UserContext:
    uid: str
    is_pro_user: bool

    async def fetch_purchases() -> list[Purchase]:
        return ...

agent = Agent[UserContext](
    ...,
)

In [ ]:
import asyncio
from dataclasses import dataclass
from agents.extensions.models.litellm_model import LitellmModel
from agents import Agent, RunContextWrapper, Runner, function_tool, set_tracing_disabled
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

@dataclass
class UserInfo:  
    name: str
    uid: int

@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo]) -> str:  
    # 可以看到，所有的工具都可以访问到这个wrapper.context
    return f"User {wrapper.context.name} is 47 years old"

async def main():
    user_info = UserInfo(name="John", uid=123)

    agent = Agent[UserInfo](  
        name="Assistant",
        tools=[fetch_user_age],
        model=llm,
    )

    result = await Runner.run(  
        starting_agent=agent,
        input="What is the age of the user?",
        context=user_info,
    )

    print(result.final_output)  
    # The user John is 47 years old.

# if __name__ == "__main__":
    # asyncio.run(main())

await main()


# 可以将 if 部分的代码替换成 await main() 吗？
# 为什么我运行代码输出结果为：The user is 47 years old.


The user is 47 years old.


## 动态指令

In [9]:
import asyncio
from dataclasses import dataclass
from agents.extensions.models.litellm_model import LitellmModel
from agents import Agent, RunContextWrapper, Runner, function_tool, set_tracing_disabled
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


@dataclass
class PatInfo:
    ip_type: str
    def advice() -> str:
        if ip_type=="发明":
            return "重点为用户讲解发明专利申请的实质审查要求"
        elif ip_type=="实用新型":
            return "重点为用户讲解实用新型专利申请的形式审查要求"

def dynamic_instructions(
    context: RunContextWrapper[PatInfo], agent: Agent[PatInfo]
) -> str:
    return f"用汉赋的句式{context.context.advice()}."


async def main():
    ip_info = PatInfo(ip_type="实用新型")

    agent = Agent[PatInfo](  
        name="Assistant",
        instructions=dynamic_instructions,
        model=llm,
    )

    result = await Runner.run(  
        starting_agent=agent,
        input="我想申请专利",
        context=ip_info,
    )

    print(result.final_output)  

# if __name__ == "__main__":
#     asyncio.run(main())

await main()

/devtool/anaconda3/envs/openai-agent/lib/python3.10/ast.py:50: RuntimeWarning: coroutine 'main' was never awaited
  return compile(source, filename, mode, flags,


TypeError: PatInfo.advice() takes 0 positional arguments but 1 was given